# Обучение ML модели для классификации полей в счетах

Этот ноутбук демонстрирует процесс обучения и валидации ML модели (RandomForest) для классификации строк текста по типам полей платежных счетов.


In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import json
import re
from pathlib import Path

# Импортируем нашу ML модель
from app.ml_model import InvoiceFieldClassifier, create_sample_training_data

# Настройка отображения
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("✅ Все библиотеки успешно импортированы")


## 1. Загрузка и подготовка обучающих данных


In [ ]:
# Генерируем обучающие данные
# В реальном проекте здесь будет загрузка из датасета с аннотированными счетами

print("Генерация обучающих данных...")
texts, labels = create_sample_training_data()

# Создаем DataFrame для анализа
df = pd.DataFrame({
    'text': texts,
    'label': labels
})

print(f"✅ Загружено записей: {len(df)}")
print(f"\nРаспределение по классам:")
print(df['label'].value_counts().sort_index())

print("\nПримеры данных:")
df.head(10)


## 2. Инициализация и обучение ML модели


In [ ]:
# Создаем классификатор
classifier = InvoiceFieldClassifier()

print("Инициализация модели...")
print(f"Типы полей для классификации: {classifier.FIELD_TYPES}")

# Обучаем модель
print("\nНачало обучения модели...")
results = classifier.train(texts, labels, test_size=0.2)

print("✅ Модель успешно обучена!")
print(f"\nРезультаты обучения:")
print(f"  - Точность (Accuracy): {results['accuracy']:.2%}")
print(f"  - Размер обучающей выборки: {results['train_size']}")
print(f"  - Размер тестовой выборки: {results['test_size']}")


## 3. Детальный анализ результатов обучения


In [ ]:
# Детальный отчет по каждому классу
report = results['classification_report']

print("Детальный отчет по классам:")
print("=" * 80)
print(f"{'Класс':<20} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
print("-" * 80)

for field_type in classifier.FIELD_TYPES:
    if field_type in report and isinstance(report[field_type], dict):
        metrics = report[field_type]
        precision = metrics.get('precision', 0)
        recall = metrics.get('recall', 0)
        f1 = metrics.get('f1-score', 0)
        support = metrics.get('support', 0)
        print(f"{field_type:<20} {precision:<12.3f} {recall:<12.3f} {f1:<12.3f} {support:<10}")

# Общие метрики
if 'macro avg' in report:
    macro = report['macro avg']
    print("-" * 80)
    print(f"{'Macro Avg':<20} {macro['precision']:<12.3f} {macro['recall']:<12.3f} {macro['f1-score']:<12.3f} {macro['support']:<10}")

if 'weighted avg' in report:
    weighted = report['weighted avg']
    print(f"{'Weighted Avg':<20} {weighted['precision']:<12.3f} {weighted['recall']:<12.3f} {weighted['f1-score']:<12.3f} {weighted['support']:<10}")


## 4. Визуализация результатов обучения


In [ ]:
# Извлекаем метрики для визуализации
field_metrics = {}
for field_type in classifier.FIELD_TYPES:
    if field_type in report and isinstance(report[field_type], dict):
        field_metrics[field_type] = {
            'precision': report[field_type]['precision'],
            'recall': report[field_type]['recall'],
            'f1-score': report[field_type]['f1-score']
        }

metrics_df = pd.DataFrame(field_metrics).T

# График метрик по классам
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, metric in enumerate(['precision', 'recall', 'f1-score']):
    ax = axes[idx]
    metrics_df[metric].plot(kind='bar', ax=ax, color='steelblue')
    ax.set_title(f'{metric.capitalize()} по классам', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric.capitalize(), fontsize=12)
    ax.set_xlabel('Классы', fontsize=12)
    ax.set_ylim(0, 1.1)
    ax.grid(axis='y', alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Матрица ошибок (Confusion Matrix)
cm = np.array(results['confusion_matrix'])
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=classifier.FIELD_TYPES,
            yticklabels=classifier.FIELD_TYPES,
            cbar_kws={'label': 'Количество'})
plt.title('Матрица ошибок (Confusion Matrix)', fontsize=14, fontweight='bold')
plt.ylabel('Истинные значения', fontsize=12)
plt.xlabel('Предсказанные значения', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


## 5. Тестирование обученной модели


In [ ]:
# Тестируем модель на примерах
test_examples = [
    ("Счет № 12345", "invoice_number"),
    ("от 15.01.2024", "date"),
    ("Поставщик: ООО \"Компания Рога и Копыта\"", "seller"),
    ("Покупатель: ИП Иванов", "buyer"),
    ("ИНН: 1234567890", "inn"),
    ("КПП: 123456789", "kpp"),
    ("Итого: 2 500 000,00", "total_amount"),
    ("НДС 20%: 66 720,35", "vat"),
    ("1. Услуга консультации - 10000.00 руб.", "item"),
    ("Адрес: г. Москва, ул. Примерная", "other"),
]

print("Тестирование модели на примерах:")
print("=" * 80)
print(f"{'Текст':<50} {'Ожидаемый':<20} {'Предсказанный':<20} {'Уверенность':<15}")
print("-" * 80)

correct = 0
for text, expected_label in test_examples:
    predicted_label, confidence = classifier.predict(text)
    is_correct = predicted_label == expected_label
    if is_correct:
        correct += 1
    
    status = "✅" if is_correct else "❌"
    print(f"{text[:48]:<50} {expected_label:<20} {predicted_label:<20} {confidence:<15.2%} {status}")

print("-" * 80)
print(f"Точность на тестовых примерах: {correct}/{len(test_examples)} ({correct/len(test_examples):.2%})")


## 6. Сохранение обученной модели


In [ ]:
# Сохраняем обученную модель
model_path = Path("../models")
model_path.mkdir(exist_ok=True)
model_file = model_path / "invoice_classifier.pkl"

print(f"Сохранение модели в {model_file}...")
classifier.save(str(model_file))
print("✅ Модель успешно сохранена!")

# Сохраняем метрики
metrics_file = model_path / "model_metrics.json"
metrics = {
    'accuracy': results['accuracy'],
    'train_size': results['train_size'],
    'test_size': results['test_size'],
    'classification_report': {k: v for k, v in report.items() if isinstance(v, dict)}
}

with open(metrics_file, 'w', encoding='utf-8') as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)
print(f"✅ Метрики сохранены в {metrics_file}")

# Проверяем загрузку модели
print("\nПроверка загрузки модели...")
test_classifier = InvoiceFieldClassifier()
test_classifier.load(str(model_file))
test_pred, test_conf = test_classifier.predict("Счет № 999")
print(f"✅ Модель успешно загружена! Тест: 'Счет № 999' -> {test_pred} (уверенность: {test_conf:.2%})")


## 7. Интеграция с системой извлечения данных


In [ ]:
# Демонстрация интеграции ML модели с системой извлечения данных
from app.data_extractor import InvoiceDataExtractor

# Загружаем обученную модель
ml_classifier = InvoiceFieldClassifier()
ml_classifier.load(str(model_file))

# Создаем экстрактор данных
extractor = InvoiceDataExtractor()

# Пример текста счета
sample_text = """
СЧЕТ № 12345
от 15.01.2024

Продавец: ООО "Компания Рога и Копыта"
ИНН: 1234567890
КПП: 123456789

Покупатель: ИП Иванов Иван Иванович

Товары и услуги:
1. Услуга консультации - 1 шт. - 10000.00 руб. - 10000.00 руб.
2. Разработка ПО - 1 шт. - 50000.00 руб. - 50000.00 руб.

Итого: 60000.00 руб.
НДС 20%: 12000.00 руб.
К оплате: 72000.00 руб.
"""

print("Демонстрация работы системы:")
print("=" * 80)

# Извлекаем данные с помощью regex
extracted_data = extractor.extract_invoice_data(sample_text)

print("\n1. Извлечение данных (Regex + ML валидация):")
print(json.dumps(extracted_data, ensure_ascii=False, indent=2))

# Используем ML для валидации извлеченных данных
print("\n2. Валидация извлеченных данных с помощью ML:")
for field, value in extracted_data.items():
    if value and isinstance(value, str):
        predicted_type, confidence = ml_classifier.predict(value)
        is_valid, val_confidence = ml_classifier.validate_extraction(field, value, sample_text)
        status = "✅" if is_valid else "⚠️"
        print(f"  {field:20s}: {value[:40]:<40} -> {predicted_type} (уверенность: {confidence:.2%}) {status}")

print("\n" + "=" * 80)
print("ИТОГОВЫЕ МЕТРИКИ МОДЕЛИ")
print("=" * 80)
print(f"\nОбщая точность ML модели: {results['accuracy']:.2%}")
print(f"\nТочность по классам:")
for field_type in classifier.FIELD_TYPES:
    if field_type in report and isinstance(report[field_type], dict):
        f1 = report[field_type]['f1-score']
        print(f"  - {field_type:20s}: F1-Score = {f1:.2%}")

print("\n" + "=" * 80)
print("\nРЕКОМЕНДАЦИИ:")
print("1. ✅ ML модель успешно обучена и интегрирована")
print("2. Увеличить датасет для обучения (больше разнообразных примеров)")
print("3. Добавить обработку различных форматов документов")
print("4. Использовать ML для валидации извлеченных данных")
print("5. Рассмотреть использование более сложных моделей (BERT, GPT) для сложных случаев")
